In [1]:
import re

In [5]:
def filter_content(text):
    # 过滤掉以 '![]' 开头的行
    # ^ 表示行开头，\Q 和 \E 用来转义 '[]' 中的特殊字符
    text = re.sub(r'\n\n!\[\][^\n]*', '', text)

    # 过滤掉被 <html> 标签包裹的内容
    # 匹配从 <html> 到 </html> 的所有内容，包括多行内容
    text = re.sub(r'<html>[\s\S]*?</html>', '', text)

    return text.strip()

def load_md(file_path, is_omit_ref_apx=True):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        sections = re.split(r'\n(?=#+ )', content)
        meta_information = {}
        title2content = {}
        for idx, sec in enumerate(sections):

                position = sec.find('\n\n')
                if position != -1 :
                        header = sec[:position]
                else:
                        match = re.search(r'^(#{1,6})[\s\S]*?(?=\n)', sec, re.MULTILINE)
                        header = match.group()
                
                if idx == 0:
                        meta_information['title'] = header
                else:
                        filtered_content = filter_content(sec[position:].strip())
                        if len(filtered_content)!=0:
                                title2content[header] = filtered_content
                
                if is_omit_ref_apx:
                        if 'conclusion' in header.lower():
                                break
        meta_information['body'] = title2content
        return meta_information

In [6]:
file_path = "/mnt/d/PycharmCode/LLMscratch/essay_searcher/data/mds/lexmatcher.md"
meta_information = load_md(file_path)


In [7]:
meta_information

{'title': '# LexMatcher: Dictionary-centric Data Collection for LLM-based Machine Translation  ',
 'body': {'## Abstract  ': 'The fine-tuning of open-source large language models (LLMs) for machine translation has recently received considerable attention, marking a shift towards data-centric research from traditional neural machine translation. However, the area of data collection for instruction finetuning in machine translation remains relatively underexplored. In this paper, we present LexMatcher, a simple yet effective method for data collection that leverages bilingual dictionaries to generate a dataset, the design of which is driven by the coverage of senses found in these dictionaries. The dataset comprises a subset retrieved from an existing corpus and a smaller synthesized subset which supplements the infrequent senses of polysemous words. Utilizing LLaMA2 as our base model, our approach outperforms the established baselines on the WMT2022 test sets and also exhibits significa

In [2]:
from pymilvus import MilvusClient

In [3]:
from unstructured.partition.pdf import partition_pdf
file_path = "/mnt/d/PycharmCode/LLMscratch/essay_searcher/pdfs/LexMatcher.pdf"
# Returns a List[Element] present in the pages of the parsed pdf document
elements = partition_pdf(file_path,  extract_images_in_pdf=False,
        infer_table_structure=True,
        chunking_strategy="by_title",
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,)

LocalEntryNotFoundError: An error happened while trying to locate the file on the Hub and we cannot find the requested files in the local cache. Please check your connection and try again or make sure your Internet connection is on.

In [57]:

# 遍历所有分块内容，并按类别组织
for el in elements:

        print(f"[{el.category}] {el.text}\n")


[Title] n u J

[Title] ] L C . s c [

[Title] Abstract

[Title] Introduction

[Title] 2 Related Work

[Title] Instructionfine-tuning

[Title] Retrieve

[Title] Corpus

[Title] Sense retrievalSense supplement

[Title] Figure 1: Illustration of our LexMatcher for instruction fine-tuning.

[Title] 3 Method

[Title] 3.1 Sense Retrieval

[Title] 3.2 Sense Supplement

[Title] 1We use unigram and bigram excluding stopwords. 2GPT-3.5-turbo-0314

[Title] Algorithm 1 Sense Retrieval in LexMatcher

[Title] 1: Input: Parallel dataset D, dictionary Φ, thresh-

[Title] old K

[Title] Lemmatize(y) then

[Title] end if

[Title] end for

[Title] end for if Found then

[Title] end if

[Title] 16: 17: end for 18: return Sr

[Title] Instruction Fine-tuning

[Title] i

[Title] Lang

[Title] Raw

[Title] Retrieval

[Title] Supplement

[Title] Zh De Ru

[Title] Table 1: The number of parallel sentences of different data sets.

[Title] 4 Experiments

[Title] 4.1 Setting

[Title] 4https://www.nltk.org/howto/wo

In [11]:
print(extracted_text)

4 2 0 2
b e F 6
] L C . s c [
2 v 4 7 6 1 1 . 9 0 3 2 : v i X r a
Published as a conference paper at ICLR 2024
A PARADIGM SHIFT IN MACHINE TRANSLATION: BOOSTING TRANSLATION PERFORMANCE OF LARGE LANGUAGE MODELS
Haoran Xu♠, Young Jin Kim♡, Amr Sharaf♡, Hany Hassan Awadalla♡
♠Johns Hopkins University, ♡Microsoft
hxu64@jhu.edu {youki,amrsharaf,hanyh}@microsoft.com
ABSTRACT
Generative Large Language Models (LLMs) have achieved remarkable advance- ments in various NLP tasks. However, these advances have not been reflected in the translation task, especially those with moderate model sizes (i.e., 7B or 13B parameters), which still lag behind conventional supervised encoder-decoder translation models. Previous studies have attempted to improve the translation capabilities of these LLMs, but their gains have been limited. In this study, we propose a novel fine-tuning approach for LLMs that is specifically designed for the translation task, eliminating the need for the abundant parallel data tha

In [13]:
import pymupdf4llm

In [46]:
file_path = "/mnt/d/PycharmCode/LLMscratch/essay_searcher/pdfs/LexMatcher.pdf"
md_text = pymupdf4llm.to_markdown(file_path)

Processing /mnt/d/PycharmCode/LLMscratch/essay_searcher/pdfs/LexMatcher.pdf...
[========================================]


In [47]:
a= md_text.split("###")

In [48]:
len(a)

15

In [55]:
a[2]

' 1 Introduction gate a principled method, LexMatcher, for curating\n\nsupervised fine-tuning data for LLM-based transla\nThe emergence of large language models (LLMs)\n\ntion. The objective is to collect a small yet carefully\n\nhas brought about new opportunities for machine\n\nselected dataset that follows a proper distribution\n\ntranslation. The modeling paradigm has gradually\n\nfor maximizing translation quality. In particular,\n\nshifted from training sequence-to-sequence mod\nwe leverage a bilingual dictionary as a pivotal re\nels from scratch to utilizing commercial or open\nsource to ensure comprehensive coverage of word\n\nsourced LLMs (Hendy et al., 2023; Agrawal et al.,\n\nor phrase senses in bilingual contexts. The con\n2023; Zhu et al., 2023; Jiao et al., 2023; Xu et al.,\n\nstruction of the dataset involves two steps: sense\n\n2024). Unlike traditional neural machine transla\nretrieval and sense supplement. In the sense re\ntion methods, which rely on abundant parallel